In [42]:
import pandas as pd
import numpy as np
import datetime

In [43]:
# data = pd.read_csv('us-counties-recent.csv', encoding = 'utf-8')
# subdata = data.loc[data.state == 'Ohio'].reset_index(drop=True)
# subdata.to_csv('oh-counties.csv', index=False, encoding='utf_8_sig')

data = pd.read_csv('us-counties.csv', on_bad_lines='skip', encoding= 'utf-8')
subdata = data.loc[data.state == 'Ohio'].reset_index(drop=True)
subdata.to_csv('oh-counties.csv', index=False, encoding='utf_8_sig')

In [44]:
subdata.head()

,date,county,state,fips,cases,deaths
0,2020-03-09,Cuyahoga,Ohio,39035.0,3.0,0.0
1,2020-03-10,Cuyahoga,Ohio,39035.0,3.0,0.0
2,2020-03-11,Cuyahoga,Ohio,39035.0,3.0,0.0
3,2020-03-11,Stark,Ohio,39151.0,1.0,0.0
4,2020-03-12,Cuyahoga,Ohio,39035.0,3.0,0.0


In [45]:
def createTableDicTwo(dicID1, dicID2, filename):
  data = pd.read_csv(filename, encoding = 'utf-8')
  dataDic = {}
  column_list = data.columns.values.tolist()
  for i in range(len(data)):
    dataID1 = data.loc[i,dicID1]
    dataID2 = data.loc[i,dicID2]
    dataID = str(dataID1) + '-' + str(dataID2)
    if(dataID in dataDic):
      dataInfo = {}
      for key in column_list:
        dataInfo[key] = data.loc[i,key]
      dataDic[dataID].append(dataInfo)
    else:
      dataInfo = {}
      for key in column_list:
        dataInfo[key] = data.loc[i,key]
      dataDic[dataID] = [dataInfo]
  return dataDic

In [41]:
county_dic = createTableDicTwo('date', 'county', 'us-counties-recent.csv')
# county_dic = createTableDicTwo('date', 'county', 'us-counties.csv')

In [46]:
county_dic['2023-02-05-Adams']

[{'date': '2023-02-05',
  'county': 'Adams',
  'state': 'Colorado',
  'fips': 8001.0,
  'cases': 164970,
  'deaths': 1459.0},
 {'date': '2023-02-05',
  'county': 'Adams',
  'state': 'Idaho',
  'fips': 16003.0,
  'cases': 807,
  'deaths': 16.0},
 {'date': '2023-02-05',
  'county': 'Adams',
  'state': 'Illinois',
  'fips': 17001.0,
  'cases': 27656,
  'deaths': 260.0},
 {'date': '2023-02-05',
  'county': 'Adams',
  'state': 'Indiana',
  'fips': 18001.0,
  'cases': 8644,
  'deaths': 120.0},
 {'date': '2023-02-05',
  'county': 'Adams',
  'state': 'Iowa',
  'fips': 19003.0,
  'cases': 931,
  'deaths': 15.0},
 {'date': '2023-02-05',
  'county': 'Adams',
  'state': 'Mississippi',
  'fips': 28001.0,
  'cases': 8513,
  'deaths': 152.0},
 {'date': '2023-02-05',
  'county': 'Adams',
  'state': 'Nebraska',
  'fips': 31001.0,
  'cases': 7758,
  'deaths': 92.0},
 {'date': '2023-02-05',
  'county': 'Adams',
  'state': 'North Dakota',
  'fips': 38001.0,
  'cases': 688,
  'deaths': 10.0},
 {'date': '20

In [47]:
def calculate_daily_cases():
  data = pd.read_csv('oh-counties.csv', encoding = 'utf-8')
  # remove unknown counties
  data = data.loc[data.county != 'unknown'].reset_index(drop=True)
  for i in range(len(data)):
    date = data.at[i,'date']
    date = date.split('-')
    date = [int(x) for x in date]
    date = datetime.date(date[0], date[1], date[2])
    # print(date)
    # print(date - datetime.timedelta(days=1))
    county = data.at[i,'county']
    previous_date = date - datetime.timedelta(days=1)
    key = str(previous_date) + '-' + county
    if(key not in county_dic):
      # print(key)
      data.at[i,'daily_cases'] = int(data.at[i,'cases'])
      data.at[i,'daily_deaths'] = int(data.at[i,'deaths'])
    else:
      case_diff = int(data.at[i,'cases'] - county_dic[key][0]['cases'])
      death_diff = int(data.at[i,'deaths'] - county_dic[key][0]['deaths'])
      if case_diff < 0:
        case_diff = np.nan
      if death_diff < 0:
        death_diff = np.nan
      data.at[i,'daily_cases'] = case_diff
      data.at[i,'daily_deaths'] = death_diff
  data.to_csv('oh-counties_updated.csv', index=False, encoding='utf_8_sig')

calculate_daily_cases()

In [48]:
def convert_osu_data():
  # parse the ohio state daily case data
  data = pd.read_csv('Daily Cases at Ohio State.csv', encoding = 'utf-8')
  data = data.drop(columns=['Semester'])
  data = data.T.reset_index()
  data = data.rename(columns={"index":"date", 0:"case"})
  data['date'] = pd.to_datetime(data['date'])
  data.to_csv('osu-cases.csv', index=False, encoding='utf_8_sig')

convert_osu_data()